In [128]:
!pip install openpyxl

Looking in indexes: http://mirrors.cloud.aliyuncs.com/pypi/simple/
     |████████████████████████████████| 249 kB 110.7 MB/s eta 0:00:01


# 加载1.8B Chat Int4模型

In [157]:
from modelscope import snapshot_download
from transformers import AutoModelForCausalLM, AutoTokenizer

model_dir = snapshot_download('qwen/Qwen-1_8B-Chat-Int4')

tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    device_map="auto",
    trust_remote_code=True
).eval()

2024-01-06 21:06:27,395 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0
Try importing flash-attention for faster inference...


# 提示词工程

In [158]:
# 城市数据
with open('city.txt','r',encoding='utf-8') as fp:
    city_list=fp.readlines()
    city_list=[line.strip().split(' ')[1] for line in city_list]

In [159]:
Q='青岛4月6日下雨么?'

prompt_template='''
给定一句话：“%s”，请你按步骤要求工作。

步骤1：识别这句话中的城市和日期共2个信息
步骤2：根据城市和日期信息，生成JSON字符串，格式为{"city":城市,"date":日期}

请问，这个JSON字符串是：
'''

# 生成SFT微调数据

Qwen的SFT数据格式要求:
```
[
  {
    "id": "identity_0",
    "conversations": [
      {
        "from": "user",
        "value": "你好"
      },
      {
        "from": "assistant",
        "value": "我是一个语言模型，我叫通义千问。"
      }
    ]
  }
]
```

In [160]:
import random
import json
import time 

Q_arr=[]
A_arr=[]

Q_list=[
    ('{city}{year}年{month}月{day}日的天气','%Y-%m-%d'),
    ('{city}{year}年{month}月{day}号的天气','%Y-%m-%d'),
    ('{city}{month}月{day}日的天气','%m-%d'),
    ('{city}{month}月{day}号的天气','%m-%d'),

    ('{year}年{month}月{day}日{city}的天气','%Y-%m-%d'),
    ('{year}年{month}月{day}号{city}的天气','%Y-%m-%d'),
    ('{month}月{day}日{city}的天气','%m-%d'),
    ('{month}月{day}号{city}的天气','%m-%d'),

    ('你们{year}年{month}月{day}日去{city}玩吗？','%Y-%m-%d'),
    ('你们{year}年{month}月{day}号去{city}玩么？','%Y-%m-%d'),
    ('你们{month}月{day}日去{city}玩吗？','%m-%d'),
    ('你们{month}月{day}号去{city}玩吗？','%m-%d'),
]

# 生成一批"1月2号"、"1月2日"、"2023年1月2号", "2023年1月2日", "2023-02-02", "03-02"之类的话术, 教会它做日期转换
for i in range(1000):
    Q=Q_list[random.randint(0,len(Q_list)-1)]
    city=city_list[random.randint(0,len(city_list)-1)]
    year=random.randint(1990,2025)
    month=random.randint(1,12)
    day=random.randint(1,28)
    time_str='{}-{}-{}'.format(year,month,day)
    date_field=time.strftime(Q[1],time.strptime(time_str,'%Y-%m-%d'))
    Q=Q[0].format(city=city,year=year,month=month,day=day) # 问题
    A=json.dumps({'city':city,'date':date_field},ensure_ascii=False)  # 回答

    Q_arr.append(prompt_template%(Q,))
    A_arr.append(A)

import pandas as pd 

df=pd.DataFrame({'Prompt':Q_arr,'Completion':A_arr})
df.to_excel('train.xlsx')

# 微调模型，生成到output_qwen

bash finetune/finetune_qlora_single_gpu.sh  -m /root/.cache/modelscope/hub/qwen/Qwen-1_8B-Chat-Int4 -d /root/Qwen/train.txt

# 加载SFT后的模型

In [161]:
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
    'output_qwen', # path to the output directory
    device_map="auto",
    trust_remote_code=True
).eval()

Try importing flash-attention for faster inference...


In [164]:
model.generation_config.top_p=0 # 只选择概率最高的token

Q_list=['2020年4月16号三亚下雨么？','青岛3-15号天气预报','5月6号下雪么，城市是威海','青岛2023年12月30号有雾霾么?','我打算6月1号去北京旅游，请问天气怎么样？','你们打算1月3号坐哪一趟航班去上海？','小明和小红是8月8号在上海结婚么?',
        '一起去东北看冰雕么，大概是1月15号左右，我们3个人一起']
for Q in Q_list:
    prompt=prompt_template%(Q,)
    A,hist=model.chat(tokenizer,prompt,history=None)
    print('Q:%s\nA:%s\n'%(Q,A))

Q:2020年4月16号三亚下雨么？
A:{"city":三亚,"date":2020-04-16}

Q:青岛3-15号天气预报
A:{"city": "青岛","date": "03-15"}

Q:5月6号下雪么，城市是威海
A:{"city":威海,"date":05-06}

Q:青岛2023年12月30号有雾霾么?
A:{"city": "青岛","date": "2023-12-30"}

Q:我打算6月1号去北京旅游，请问天气怎么样？
A:{"city": "北京","date": "06-01"}

Q:你们打算1月3号坐哪一趟航班去上海？
A:{"city": "上海","date": "01-03"}

Q:小明和小红是8月8号在上海结婚么?
A:{"city": "上海","date": "08-08"}

Q:一起去东北看冰雕么，大概是1月15号左右，我们3个人一起
A:{"city": "东北", "date": "01-15"}



In [ ]:
prompt='青岛海边钓鱼需要特别注意什么？'
resp,hist=model.chat(tokenizer,prompt,history=None)
print(resp)